In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'chest-xray-pneumonia' dataset.
Path to dataset files: /kaggle/input/chest-xray-pneumonia


In [4]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report,confusion_matrix


In [5]:
IMG_SIZE=(224,224)
BATCH_SIZE=32
train_dir='/kaggle/input/chest-xray-pneumonia/chest_xray/train'
val_dir='/kaggle/input/chest-xray-pneumonia/chest_xray/val'
test_dir='/kaggle/input/chest-xray-pneumonia/chest_xray/test'

In [ ]:
train_datagen=ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.1,
    horizontal_flip=True,
)
val_test_datagen=ImageDataGenerator(rescale=1./255)
train_gen=train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)
val_gen=val_test_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)
test_gen=val_test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [7]:
base_model=VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(224,224,3)
)
for layer in base_model.layers:
  layer.trainable=False


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [9]:
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(128,activation='relu')(x)
x=Dropout(0.5)(x)
output=Dense(1,activation='sigmoid')(x)
model=Model(inputs=base_model.input,outputs=output)
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [10]:
history=model.fit(train_gen,
                  validation_data=val_gen,
                  epochs=5)
model.save("pneumonia_vgg16_model.h5")
print("model saved successfully")

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 159s 841ms/step - accuracy: 0.7090 - loss: 0.5783 - val_accuracy: 0.5000 - val_loss: 0.7917
Epoch 2/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 110s 674ms/step - accuracy: 0.7666 - loss: 0.4685 - val_accuracy: 0.5000 - val_loss: 0.7297
Epoch 3/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 109s 665ms/step - accuracy: 0.8150 - loss: 0.3896 - val_accuracy: 0.5625 - val_loss: 0.6828
Epoch 4/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 108s 661ms/step - accuracy: 0.8570 - loss: 0.3393 - val_accuracy: 0.6250 - val_loss: 0.6455
Epoch 5/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 108s 662ms/step - accuracy: 0.8852 - loss: 0.3051 - val_accuracy: 0.6875 - val_loss: 0.6215


model saved successfully
